<a href="https://www.kaggle.com/code/muhammadmetwally/data-science-project?scriptVersionId=175805782" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

BigMart Sales Prediction practice problem
We have train (8523) , train data set has both input and output variable(s). We need to predict the sales for test data set.

Item_Identifier: Unique product ID

Item_Weight: Weight of product

Item_Fat_Content: Whether the product is low fat or not
    
Item_Visibility: The % of total display area of all products in a store allocated to the particular product

Item_Type: The category to which the product belongs

Item_MRP: Maximum Retail Price (list price) of the product

Outlet_Identifier: Unique store ID

Outlet_Establishment_Year: The year in which store was established

Outlet_Size: The size of the store in terms of ground area covered

Outlet_Location_Type: The type of city in which the store is located

Outlet_Type: Whether the outlet is just a grocery store or some sort of supermarket

Item_Outlet_Sales: Sales of the product in the particulat store. This is the outcome variable to be predicted

# Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
import plotly.graph_objects as go
from xgboost import XGBRegressor
import plotly.express as px

# Data collection and analysis

In [2]:
big_mart_data = pd.read_csv("/kaggle/input/big-mart-sales/Train.csv")
# big_mart_data=pd.concat([big_mart_data,pd.read_csv("D:\\04.1Ultrasonic_Sensor\\archive (9) 2\\Test_modified.csv")[:1504]])
# big_mart_data=pd.concat([big_mart_data,pd.read_csv("D:\\04.1Ultrasonic_Sensor\\archive (9) 2\\Test.csv")[:1504]])

In [3]:
# first 5 rows of the dataframe
big_mart_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
big_mart_data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [5]:
# number of data points & number of features
big_mart_data.shape

(8523, 12)

In [6]:
# getting some information about thye dataset
big_mart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [7]:
# checking for missing values
big_mart_data.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [8]:
#Box Plot suggest we dont have any outlier and hence we can change missing values with 'Mean'
fig = px.box(big_mart_data, y='Item_Weight', title='Item Weight Distribution')
fig.update_layout(width=800, height=500)  
fig.show()

In [9]:
# mean value of "Item_Weight" column
big_mart_data['Item_Weight'].mean()

12.857645184135976

In [10]:
# filling the missing values in "Item_weight column" with "Mean" value
big_mart_data['Item_Weight'].fillna(big_mart_data['Item_Weight'].mean(), inplace=True)

/tmp/ipykernel_18/2509980927.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [11]:
big_mart_data.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [12]:

big_mart_data['Outlet_Size'].mode()

0    Medium
Name: Outlet_Size, dtype: object

In [13]:
# Count occurrences of each combination
counts = big_mart_data.groupby(['Outlet_Size', 'Outlet_Type']).size().unstack(fill_value=0)
print(counts)
# Plot using Plotly
fig = px.bar(counts, barmode='stack')

# Customize the layout
fig.update_layout(
    title='Outlet Type vs Outlet Size',
    xaxis_title='Outlet Type',
    yaxis_title='Count',
    xaxis={'categoryorder': 'total descending'},  # Sort categories by total count
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    margin=dict(l=50, r=50, t=50, b=50)  # Add margin for better spacing
)

# Show the plot
fig.show()


Outlet_Type  Grocery Store  Supermarket Type1  Supermarket Type2  \
Outlet_Size                                                        
High                     0                932                  0   
Medium                   0                930                928   
Small                  528               1860                  0   

Outlet_Type  Supermarket Type3  
Outlet_Size                     
High                         0  
Medium                     935  
Small                        0  


/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [14]:
mode_of_outlet_size = big_mart_data.pivot_table(values='Outlet_Size',columns='Outlet_Type',aggfunc=lambda x: x.mode()[0])

In [15]:
mode_of_outlet_size

Outlet_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Outlet_Size,Small,Small,Medium,Medium


In [16]:
miss_values = big_mart_data['Outlet_Size'].isnull()  
miss_values

0       False
1       False
2       False
3        True
4       False
        ...  
8518    False
8519     True
8520    False
8521    False
8522    False
Name: Outlet_Size, Length: 8523, dtype: bool

In [17]:
# filling the missing values in "Outlet_Size" column with Mode
#Here we take Outlet_Size column & Outlet_Type column since they are correlated
big_mart_data.loc[miss_values, 'Outlet_Size'] = big_mart_data.loc[miss_values,'Outlet_Type'].apply(lambda x: mode_of_outlet_size[x])

In [18]:
# checking for missing values
big_mart_data.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

# Data Visualization

In [19]:
# Create a histogram trace
histogram_trace = go.Histogram(x=big_mart_data['Item_Weight'], 
                               marker=dict(color='blue', opacity=0.5))

# Create layout
layout = go.Layout(title='Item Weight Distribution',
                   xaxis=dict(title='Item Weight'),
                   yaxis=dict(title='Frequency'))

# Create figure
fig = go.Figure(data=[histogram_trace], layout=layout)

# Show the plot
fig.show()


In [20]:
#Box Plot suggest we dont have any outlier and hence we can change missing values with 'Mean'
fig = px.box(big_mart_data, y='Item_Weight', title='Item Weight Distribution')
fig.update_layout(width=800, height=500)  
fig.show()

In [21]:
# Create a histogram trace
histogram_trace = go.Histogram(x=big_mart_data['Item_Visibility'], 
                               marker=dict(color='green', opacity=0.5))

# Create layout
layout = go.Layout(title='Item Visibility Distribution',
                   xaxis=dict(title='Item Visibility'),
                   yaxis=dict(title='Frequency')) 

# Create figure
fig = go.Figure(data=[histogram_trace], layout=layout)

# Show the plot
fig.show()



In [22]:
big_mart_data.loc[big_mart_data['Item_Visibility'] == 0, 'Item_Visibility'] = big_mart_data['Item_Visibility'].mean()

In [23]:
big_mart_data['Item_Visibility'] = np.log(1+big_mart_data['Item_Visibility'])
# Create a histogram trace
histogram_trace = go.Histogram(x=big_mart_data['Item_Visibility'], 
                               marker=dict(color='green', opacity=0.5))
# Create layout
layout = go.Layout(title='Item Visibility Distribution',
                   xaxis=dict(title='Item Visibility'),
                   yaxis=dict(title='Frequency')) 
# Create figure
fig = go.Figure(data=[histogram_trace], layout=layout)

# Show the plot
fig.show()

In [24]:
# Plotting with Plotly
fig = go.Figure()

# Adding histogram
fig.add_trace(go.Histogram(x=big_mart_data['Item_MRP'], name='Histogram'))



# Layout settings
fig.update_layout(title='Distribution of Item_MRP',
                  xaxis_title='Item_MRP',
                  yaxis_title='Frequency')

fig.show()

In [25]:
# Plotting with Plotly
fig = go.Figure()

# Adding histogram
fig.add_trace(go.Histogram(x=big_mart_data['Item_Outlet_Sales'], name='Histogram'))



# Layout settings
fig.update_layout(title='Distribution of Item_Outlet_Sales',
                  xaxis_title='Item_Outlet_Sales',
                  yaxis_title='Frequency')

fig.show()

In [26]:
big_mart_data.drop(big_mart_data[big_mart_data['Item_Outlet_Sales'] <= 0].index, inplace=True)
big_mart_data['Item_Outlet_Sales'].fillna(big_mart_data['Item_Outlet_Sales'].mean(), inplace=True)
big_mart_data['Item_Outlet_Sales'] = np.log(1+big_mart_data['Item_Outlet_Sales'])
#big_mart_data['Item_Outlet_Sales'] = np.exp(big_mart_data['Item_Outlet_Sales']) - 1 while reverse to orgiinal

/tmp/ipykernel_18/2315809334.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [27]:
# Plotting with Plotly
fig = go.Figure()

# Adding histogram
fig.add_trace(go.Histogram(x=big_mart_data['Item_Outlet_Sales'], name='Histogram'))



# Layout settings
fig.update_layout(title='Distribution of Item_Outlet_Sales',
                  xaxis_title='Item_Outlet_Sales',
                  yaxis_title='Frequency')

fig.show()

In [28]:
big_mart_data.shape

(8523, 12)

In [29]:
import plotly.express as px

# Create a count plot using Plotly Express
fig = px.histogram(big_mart_data, x='Outlet_Establishment_Year',color_discrete_sequence=px.colors.qualitative.Pastel, title='Outlet Establishment Year Distribution')

# Update layout
fig.update_layout(xaxis_title='Outlet Establishment Year', yaxis_title='Count', 
                  xaxis=dict(type='category'), barmode='group')
# Sort the x-axis categories
fig.update_xaxes(categoryorder='category ascending')


# Show the plot
fig.show()

In [30]:
import plotly.express as px

# Create a count plot using Plotly Express
fig = px.histogram(big_mart_data, x='Item_Fat_Content')

# Update layout
fig.update_layout(xaxis_title='Item_Fat_Content', yaxis_title='Count', 
                  xaxis=dict(type='category'), barmode='group')

# Show the plot
fig.show()

In [31]:
import plotly.express as px

# Create a count plot using Plotly Express
fig = px.histogram(big_mart_data, x='Item_Type')

# Update layout
fig.update_layout(xaxis_title='Item_Type', yaxis_title='Count', 
                  xaxis=dict(type='category'), barmode='group')

# Show the plot
fig.show()

In [32]:


# Create the scatter plot using Plotly
fig = go.Figure(data=go.Scatter(x=big_mart_data['Item_Outlet_Sales'],
                                y=big_mart_data['Item_Visibility'],
                                mode='markers'))

# Update layout
fig.update_layout(title='Item Visibility vs. Item Outlet Sales',
                  xaxis_title='Item Outlet Sales',
                  yaxis_title='Item Visibility')

# Show the plot
fig.show()


In [33]:


# Create the scatter plot using Plotly
fig = go.Figure(data=go.Scatter(x=big_mart_data['Item_Outlet_Sales'],
                                y=big_mart_data['Item_MRP'],
                                mode='markers'))

# Update layout
fig.update_layout(title='Item MRP vs. Item Outlet Sales',
                  xaxis_title='Item Outlet Sales',
                  yaxis_title='Item MRP')

# Show the plot
fig.show()

# Label Encoding

In [34]:
big_mart_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.015920,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,8.225808
1,DRC01,5.92,Regular,0.019095,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,6.096776
2,FDN15,17.50,Low Fat,0.016621,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,7.648868
3,FDX07,19.20,Regular,0.064037,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,6.597664
4,NCD19,8.93,Low Fat,0.064037,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,6.903451


In [35]:
big_mart_data['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64

In [36]:
big_mart_data.replace({'Item_Fat_Content': {'low fat':'Low Fat','LF':'Low Fat', 'reg':'Regular'}}, inplace=True)

In [37]:
big_mart_data['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64

In [38]:
encoder = LabelEncoder()

In [39]:
big_mart_data['Item_Identifier'] = encoder.fit_transform(big_mart_data['Item_Identifier'])

big_mart_data['Item_Fat_Content'] = encoder.fit_transform(big_mart_data['Item_Fat_Content'])

big_mart_data['Item_Type'] = encoder.fit_transform(big_mart_data['Item_Type'])

big_mart_data['Outlet_Identifier'] = encoder.fit_transform(big_mart_data['Outlet_Identifier'])

big_mart_data['Outlet_Size'] = encoder.fit_transform(big_mart_data['Outlet_Size'])

big_mart_data['Outlet_Location_Type'] = encoder.fit_transform(big_mart_data['Outlet_Location_Type'])

big_mart_data['Outlet_Type'] = encoder.fit_transform(big_mart_data['Outlet_Type'])

In [40]:
big_mart_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,156,9.30,0,0.015920,4,249.8092,9,1999,1,0,1,8.225808
1,8,5.92,1,0.019095,14,48.2692,3,2009,1,2,2,6.096776
2,662,17.50,0,0.016621,10,141.6180,9,1999,1,0,1,7.648868
3,1121,19.20,1,0.064037,6,182.0950,0,1998,2,2,0,6.597664
4,1297,8.93,0,0.064037,9,53.8614,1,1987,0,2,1,6.903451


Splitting features and Target

In [41]:
X = big_mart_data.drop(columns='Item_Outlet_Sales', axis=1)
Y = big_mart_data['Item_Outlet_Sales']

In [42]:
X

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,156,9.300,0,0.015920,4,249.8092,9,1999,1,0,1
1,8,5.920,1,0.019095,14,48.2692,3,2009,1,2,2
2,662,17.500,0,0.016621,10,141.6180,9,1999,1,0,1
3,1121,19.200,1,0.064037,6,182.0950,0,1998,2,2,0
4,1297,8.930,0,0.064037,9,53.8614,1,1987,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
8518,370,6.865,0,0.055230,13,214.5218,1,1987,0,2,1
8519,897,8.380,1,0.045912,0,108.1570,7,2002,2,1,1
8520,1357,10.600,0,0.034581,8,85.1224,6,2004,2,1,1
8521,681,7.210,1,0.135597,13,103.1332,3,2009,1,2,2


In [43]:
Y

0       8.225808
1       6.096776
2       7.648868
3       6.597664
4       6.903451
          ...   
8518    7.929984
8519    6.310436
8520    7.085159
8521    7.521100
8522    6.642056
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

In [44]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=2)

In [45]:
print(X.shape, X_train.shape, X_test.shape)

(8523, 11) (5966, 11) (2557, 11)


# Machine Learning Model for regression

# XGBoost Regressor

In [46]:
regressor = XGBRegressor()

In [47]:
regressor.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [48]:
pd.DataFrame({
    'feature':X.columns,
    'XGBRF_importance':regressor.fit(X_train, Y_train).feature_importances_
    
}).sort_values(by='XGBRF_importance',ascending=False)

,feature,XGBRF_importance
10,Outlet_Type,0.926887
5,Item_MRP,0.030763
7,Outlet_Establishment_Year,0.011626
9,Outlet_Location_Type,0.005021
2,Item_Fat_Content,0.004160
1,Item_Weight,0.003932
3,Item_Visibility,0.003730
8,Outlet_Size,0.003663
4,Item_Type,0.003603
6,Outlet_Identifier,0.003349


# Evaluation

In [49]:
# prediction on training data
training_data_prediction = regressor.predict(X_train)

In [50]:
# R squared Value
r2_train = metrics.r2_score(Y_train, training_data_prediction)

In [51]:
print('R Squared value = ', r2_train)

R Squared value =  0.9122228652863242


In [52]:
# prediction on test data
test_data_prediction = regressor.predict(X_test)

In [53]:
from sklearn.model_selection import cross_val_score
# R squared Value
r2_test = metrics.r2_score(Y_test, test_data_prediction)

# predict the training set
pred = regressor.predict(X)

# perform cross-validation
cv_score = cross_val_score(regressor, X, Y, scoring='r2', cv=5)
cv_score = np.abs(np.mean(cv_score))

print("Model Report")
print("CV Score:", cv_score)

Model Report
CV Score: 0.6845027640882337


In [54]:
print('R Squared value = ', r2_test)

R Squared value =  0.7008335597040496


# Ridge

In [55]:
from sklearn.linear_model import Ridge
regressor = Ridge(alpha=0.1)  # alpha is the regularization parameter
regressor.fit(X_train, Y_train)

Ridge(alpha=0.1)

# Evaluation

In [56]:
# prediction on training data
training_data_prediction = regressor.predict(X_train)

In [57]:
# R squared Value
r2_train = metrics.r2_score(Y_train, training_data_prediction)

In [58]:
print('R Squared value = ', r2_train)

R Squared value =  0.5744924232148398


In [59]:
# prediction on test data
test_data_prediction = regressor.predict(X_test)

In [60]:
# R squared Value
r2_test = metrics.r2_score(Y_test, test_data_prediction)

# predict the training set
pred = regressor.predict(X)

# perform cross-validation
cv_score = cross_val_score(regressor, X, Y, scoring='r2', cv=5)
cv_score = np.abs(np.mean(cv_score))

print("Model Report")
print("CV Score:", cv_score)

Model Report
CV Score: 0.5780129259634664


In [61]:
print('R Squared value = ', r2_test)

R Squared value =  0.5915008937950537


# Classification

In [62]:
big_mart_data['Item_Fat_Content'].unique()

array([0, 1])

In [63]:
big_mart_data["Outlet_Size"].unique()

array([1, 2, 0])

In [64]:
big_mart_data['Outlet_Type'].unique()

array([1, 2, 0, 3])

In [65]:
big_mart_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,156,9.30,0,0.015920,4,249.8092,9,1999,1,0,1,8.225808
1,8,5.92,1,0.019095,14,48.2692,3,2009,1,2,2,6.096776
2,662,17.50,0,0.016621,10,141.6180,9,1999,1,0,1,7.648868
3,1121,19.20,1,0.064037,6,182.0950,0,1998,2,2,0,6.597664
4,1297,8.93,0,0.064037,9,53.8614,1,1987,0,2,1,6.903451


In [66]:
# Create correlation matrix
corr_matrix = big_mart_data.corr()

# Create heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
                   z=corr_matrix.values,
                   x=corr_matrix.columns,
                   y=corr_matrix.index,
                   colorscale='magma',
                   colorbar=dict(title='Correlation'),
                   zmax=0.9,  # Set max value for color scale
                   zmin=-0.9  # Set min value for color scale
                 ))

# Update layout
fig.update_layout(
    title='Correlation Heatmap',
    width=800,
    height=800,
    xaxis_title='Features',
    yaxis_title='Features',
    margin=dict(l=50, r=50, t=50, b=50)
)

fig.show()

In [67]:
import plotly.express as px

# Group by 'Item_Fat_Content' and calculate the size of each group
pie_data = big_mart_data.groupby('Item_Fat_Content').size().reset_index(name='count')

# Create a pie plot using Plotly Express
fig = px.pie(pie_data, values='count', names='Item_Fat_Content', 
             title='Item Fat Content Distribution', 
             labels={'Item_Fat_Content': 'Fat Content'})

# Show the plot
fig.show()


In [68]:
# Create countplot using Plotly Express
fig = px.histogram(big_mart_data['Item_Fat_Content'], x='Item_Fat_Content')
# Update layout
fig.update_layout(
    title='Balanced Item Fat Content',
    xaxis_title='Item Fat Content',
    yaxis_title='Count',
    bargap=0.1,
    margin=dict(l=50, r=50, t=50, b=50),
    width=400,
    height=400
)
# Update x-axis tick values
fig.update_xaxes(tickvals=[0, 1])
fig.show()

In [69]:
from imblearn.over_sampling import RandomOverSampler

In [70]:
X = big_mart_data.drop("Item_Fat_Content", axis = 1)
y = big_mart_data['Item_Fat_Content']

In [71]:
oversample = RandomOverSampler(sampling_strategy='minority')

X_resampled, y_resampled = oversample.fit_resample(X, y)

In [72]:
X_resampled.shape

(11034, 11)

In [73]:
new_df_balanced = pd.concat([X_resampled, y_resampled], axis = 1)

In [74]:
new_df_balanced.shape

(11034, 12)

In [75]:
# Create countplot using Plotly Express
fig = px.histogram(new_df_balanced, x='Item_Fat_Content')
# Update layout
fig.update_layout(
    title='Balanced Item Fat Content',
    xaxis_title='Item Fat Content',
    yaxis_title='Count',
    bargap=0.1,
    margin=dict(l=50, r=50, t=50, b=50),
    width=400,
    height=400
)
# Update x-axis tick values
fig.update_xaxes(tickvals=[0, 1])
fig.show()

In [76]:
X = new_df_balanced[["Item_Outlet_Sales","Item_MRP","Item_Weight",'Item_Visibility']]
y = new_df_balanced['Item_Fat_Content']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

In [77]:
print("X_train",X_train.shape)
print("y_train",y_train.shape)
print("X_test",X_test.shape)
print("y_test",y_test.shape)

X_train (7723, 4)
y_train (7723,)
X_test (3311, 4)
y_test (3311,)


# RandomForestClassifier

In [78]:
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier()
model3.fit(X_train, y_train)

RandomForestClassifier()

In [79]:
y_pred3 = model3.predict(X_test)
y_pred_prob = model3.predict_proba(X_test)[:, 1]

In [80]:
from sklearn.metrics import confusion_matrix
cm2 = confusion_matrix(y_test, y_pred3)
cm2

array([[1502,  130],
       [ 113, 1566]])

In [81]:
fig = go.Figure(data=go.Heatmap(
                   z=cm2,
                   colorscale='Blues',  # Use 'Blues' colormap
                   hoverongaps=False
                 ))

# Update layout
fig.update_layout(
    title='Confusion Matrix',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    width=400,
    height=300,
    margin=dict(l=50, r=50, t=50, b=50),
    xaxis_showgrid=False  # Hide horizontal grid lines
)

# Update x-axis tick values
fig.update_xaxes(tickvals=[0, 1])

# Update x-axis tick values
fig.update_yaxes(tickvals=[0, 1])

fig.show()

# ROC and AUC

In [82]:
from sklearn.metrics import roc_curve,auc

# Assuming y_test contains the true labels and y_pred contains the predicted probabilities for the positive class

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Compute ROC AUC score
roc_auc = auc(fpr,tpr)

# Plot ROC curve with Plotly
fig = go.Figure()

# Add ROC curve
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', line=dict(color='blue', width=2), name=f'ROC curve (AUC = {roc_auc:.2f})'))

# Add diagonal line
fig.add_shape(type='line', x0=0, y0=0, x1=1, y1=1, line=dict(color='gray', dash='dash'))

# Update layout
fig.update_layout(
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    title='Receiver Operating Characteristic (ROC) Curve',
    legend=dict(x=0.98, y=0.02, bgcolor='rgba(255, 255, 255, 0.5)'),
    margin=dict(l=50, r=50, t=50, b=50),
    width=800,
    height=600
)

# Add text annotation for AUC value
fig.add_annotation(
    x=0.5,
    y=0.5,
    text=f'AUC = {roc_auc:.2f}',
    showarrow=False,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="black"
    )
)

fig.show()




In [83]:
from sklearn.metrics import accuracy_score
print("Accuracy Score of RandomForestClassifier is ",accuracy_score(y_test, y_pred3))

Accuracy Score of RandomForestClassifier is  0.9266082754454847


In [84]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      1632
           1       0.92      0.93      0.93      1679

    accuracy                           0.93      3311
   macro avg       0.93      0.93      0.93      3311
weighted avg       0.93      0.93      0.93      3311



In [85]:
y_pred3=model3.predict(X)
# perform cross-validation
cv_score = cross_val_score(model3, X, y, scoring='accuracy', cv=5)
cv_score = np.abs(np.mean(cv_score))

print("Model Report")

print("CV Score:", cv_score)

Model Report
CV Score: 0.9330267865248667


# LogisticRegression 

In [86]:
from sklearn.linear_model import LogisticRegression
model3 = LogisticRegression()
model3.fit(X_train, y_train)

LogisticRegression()

In [87]:
y_pred3 = model3.predict(X_test)
y_pred_prob = model3.predict_proba(X_test)[:, 1]

In [88]:
cm2 = confusion_matrix(y_test, y_pred3)
cm2

array([[953, 679],
       [919, 760]])

In [89]:
fig = go.Figure(data=go.Heatmap(
                   z=cm2,
                   colorscale='Blues',  # Use 'Blues' colormap
                   hoverongaps=False
                 ))

# Update layout
fig.update_layout(
    title='Confusion Matrix',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    width=400,
    height=300,
    margin=dict(l=50, r=50, t=50, b=50),
    xaxis_showgrid=False  # Hide horizontal grid lines
)

# Update x-axis tick values
fig.update_xaxes(tickvals=[0, 1])

# Update x-axis tick values
fig.update_yaxes(tickvals=[0, 1])

fig.show()

In [90]:
from sklearn.metrics import roc_curve,auc

# Assuming y_test contains the true labels and y_pred contains the predicted probabilities for the positive class

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Compute ROC AUC score
roc_auc = auc(fpr,tpr)

# Plot ROC curve with Plotly
fig = go.Figure()

# Add ROC curve
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', line=dict(color='blue', width=2), name=f'ROC curve (AUC = {roc_auc:.2f})'))

# Add diagonal line
fig.add_shape(type='line', x0=0, y0=0, x1=1, y1=1, line=dict(color='gray', dash='dash'))

# Update layout
fig.update_layout(
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    title='Receiver Operating Characteristic (ROC) Curve',
    legend=dict(x=0.98, y=0.02, bgcolor='rgba(255, 255, 255, 0.5)'),
    margin=dict(l=50, r=50, t=50, b=50),
    width=800,
    height=600
)

# Add text annotation for AUC value
fig.add_annotation(
    x=0.5,
    y=0.5,
    text=f'AUC = {roc_auc:.2f}',
    showarrow=False,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="black"
    )
)

fig.show()




In [91]:
print("Accuracy Score of LogisticRegression is ",accuracy_score(y_test, y_pred3))

Accuracy Score of LogisticRegression is  0.5173663545756569


In [92]:
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.51      0.58      0.54      1632
           1       0.53      0.45      0.49      1679

    accuracy                           0.52      3311
   macro avg       0.52      0.52      0.52      3311
weighted avg       0.52      0.52      0.52      3311



In [93]:
y_pred3=model3.predict(X)
# perform cross-validation
cv_score = cross_val_score(model3, X, y, scoring='accuracy', cv=5)
cv_score = np.abs(np.mean(cv_score))

print("Model Report")

print("CV Score:", cv_score)

Model Report
CV Score: 0.5222957038122745


# Clustring

In [94]:
from sklearn.cluster import KMeans
X = X[['Item_Visibility', 'Item_MRP']].values 
# Item_MRP
k = 10
kmeans = KMeans(n_clusters=k)
kmeans.fit(X)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
print("Labels: ",labels)
print("centroids: ",centroids)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Labels:  [6 2 9 ... 3 2 8]
centroids:  [[6.38298321e-02 1.22751816e+02]
 [6.69446814e-02 1.90064362e+02]
 [7.29265551e-02 4.12600035e+01]
 [6.91336378e-02 2.22695463e+02]
 [6.52299728e-02 8.49418029e+01]
 [6.84948713e-02 1.70810044e+02]
 [6.83161908e-02 2.52943278e+02]
 [6.97906947e-02 5.74810582e+01]
 [6.65800887e-02 1.04034024e+02]
 [6.87170995e-02 1.49603937e+02]]


In [95]:
# finding The Within-Cluster Sum of Squares value for different number of clusters

wcss = []

for i in range(1,11):
  kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
  kmeans.fit(X)

  wcss.append(kmeans.inertia_)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

In [96]:
x_values = list(range(1, 11))

trace = go.Scatter(
    x=x_values,  # x-axis values (Number of Clusters)
    y=wcss,  # y-axis values (WCSS)
    mode='lines+markers',
    marker=dict(color='blue'),  # Marker color
    name='WCSS'  # Name for legend
)

# Create the layout
layout = go.Layout(
    title='Elbow Method for Optimal Number of Clusters',
    xaxis=dict(title='Number of Clusters'),  # x-axis label
    yaxis=dict(title='WCSS'),  # y-axis label
)

# Create the figure
fig = go.Figure(data=[trace], layout=layout)

# Show the plot
fig.show()


In [97]:
kmeans = KMeans(n_clusters=4, init='k-means++', random_state=0)

# return a label for each data point based on their cluster
Y = kmeans.fit_predict(X)
X



/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



array([[1.59199042e-02, 2.49809200e+02],
       [1.90947454e-02, 4.82692000e+01],
       [1.66211748e-02, 1.41618000e+02],
       ...,
       [3.04372658e-02, 2.27635200e+02],
       [2.62328414e-02, 3.47532000e+01],
       [8.15159727e-03, 1.01267400e+02]])

In [98]:
# Define colors for clusters
colors = ['green', 'red', 'yellow', 'violet']

# Initialize a figure
fig = go.Figure()

# Add traces for each cluster
for i in range(len(colors)):
    fig.add_trace(go.Scatter(
        x=X[Y == i, 0],  # X coordinates for points in cluster i
        y=X[Y == i, 1],  # Y coordinates for points in cluster i
        mode='markers',
        marker=dict(color=colors[i], size=10),
        name=f'Cluster {i + 1}'
    ))

# Add centroids as scatter plot
fig.add_trace(go.Scatter(
    x=kmeans.cluster_centers_[:, 0],  # X coordinates for centroids
    y=kmeans.cluster_centers_[:, 1],  # Y coordinates for centroids
    mode='markers',
    marker=dict(color='cyan', size=12, symbol='x'),
    name='Centroids'
))

# Update layout
fig.update_layout(
    title='Clusters and Centroids',
    xaxis_title='Item Visibility',
    yaxis_title='Item MRP',
    showlegend=True
)

# Show plot
fig.show()